In [30]:
%pip install openai
%pip install langchain
%pip install youtube_search
%python -m pip install pytube
%pip install pydub

     |████████████████████████████████| 57 kB 3.1 MB/s eta 0:00:011


In [32]:
import os
import ast
import openai
from langchain.tools import YouTubeSearchTool
from pytube import YouTube
from pydub import AudioSegment

In [48]:
os.environ["OPENAI_API_KEY"] = "..."

raw_output_folder = "audio/raw"
segments_output_folder = "audio/segments"
output_file_basename = "audio_segment"

In [54]:
def download_audio(yt_url, output_folder, output_filename):
    youtube = YouTube(yt_url)
    stream = youtube.streams.filter(only_audio=True).first()
    stream.download(output_path=output_folder, filename=f"{output_filename}.mp4")
    audio = AudioSegment.from_file(f"{output_folder}/{output_filename}.mp4")
    audio.export(f"{output_folder}/{output_filename}.mp3", format="mp3")
    os.remove(f"{output_folder}/{output_filename}.mp4")
    return audio

In [4]:
tool = YouTubeSearchTool()

In [55]:
youtube_links = tool.run("asmr lectura susurrada, 100")
youtube_links = ast.literal_eval(youtube_links)
youtube_links

['/watch?v=Z8wkoY_WWTc&pp=ygUWYXNtciBsZWN0dXJhIHN1c3VycmFkYQ%3D%3D',
 '/watch?v=meEyLOBozhA&pp=ygUWYXNtciBsZWN0dXJhIHN1c3VycmFkYQ%3D%3D',
 '/watch?v=BhNTF2n7LDI&pp=ygUWYXNtciBsZWN0dXJhIHN1c3VycmFkYQ%3D%3D',
 '/watch?v=Qko7rSNwEKE&pp=ygUWYXNtciBsZWN0dXJhIHN1c3VycmFkYQ%3D%3D',
 '/watch?v=Hok30oLmaCA&pp=ygUWYXNtciBsZWN0dXJhIHN1c3VycmFkYQ%3D%3D',
 '/watch?v=sUfClIGTP5Y&pp=ygUWYXNtciBsZWN0dXJhIHN1c3VycmFkYQ%3D%3D',
 '/watch?v=7yEFOiT0QJI&pp=ygUWYXNtciBsZWN0dXJhIHN1c3VycmFkYQ%3D%3D',
 '/watch?v=clw9gronjgs&pp=ygUWYXNtciBsZWN0dXJhIHN1c3VycmFkYQ%3D%3D',
 '/watch?v=vvI8pf4bxFk&pp=ygUWYXNtciBsZWN0dXJhIHN1c3VycmFkYQ%3D%3D',
 '/watch?v=eGz7ifSpb-s&pp=ygUWYXNtciBsZWN0dXJhIHN1c3VycmFkYQ%3D%3D',
 '/watch?v=y6nwyuYZ1DE&pp=ygUWYXNtciBsZWN0dXJhIHN1c3VycmFkYQ%3D%3D',
 '/watch?v=XzITIglBvfQ&pp=ygUWYXNtciBsZWN0dXJhIHN1c3VycmFkYQ%3D%3D',
 '/watch?v=n-XkdvZXfn0&pp=ygUWYXNtciBsZWN0dXJhIHN1c3VycmFkYQ%3D%3D',
 '/watch?v=_swpGj1Lx2Q&pp=ygUWYXNtciBsZWN0dXJhIHN1c3VycmFkYQ%3D%3D',
 '/watch?v=G0CMOlxBBG0&pp=ygUWYXNt

In [56]:
len(youtube_links)

20

In [57]:
audio_segment_duration_seconds = 4

for i in range(len(youtube_links)):
    yt_url = youtube_links[i]
    yt_url = f"https://www.youtube.com{yt_url}"
    output_filename = f"{output_file_basename}_{i}"
    audio = download_audio(yt_url, raw_output_folder, output_filename)

    # split into 4-second segments and save
    shift_milliseconds = 60*1000
    segment_len_milliseconds = audio_segment_duration_seconds*1000
    for j in range(shift_milliseconds, len(audio), segment_len_milliseconds):  # start at <shift_seconds> milliseconds and step by <segment_len_milliseconds> milliseconds
        segment = audio[j:j+segment_len_milliseconds]
        num = f"{i}_{j}"
        segment.export(f"{segments_output_folder}/{output_filename}_{num}.mp3", format="mp3")

In [58]:
# list all the files in the directory
files = os.listdir(segments_output_folder)

# count the number of files
num_files = sum([1 for file in files if os.path.isfile(os.path.join(segments_output_folder, file))])

print(f"There are {num_files} files in the folder.")

There are 8239 files in the folder.
